# Bias Uncovering Test Case (BTC) for Gender Bias

In [7]:
import os
import pandas as pd
import numpy as np
import math
import time
import pickle

from metric import evaluate_btc

## Measuring the Performance of the Fine-tuned SA models on the Test set

In [8]:
def load_pickle(fpath):
    with open(fpath, 'rb') as f:
        pred = pickle.load(f)
    return pred

def accuracy(label, prediction):
    return 100 * sum(label == prediction) / len(label)

def calculate_test_accuracy(task, model) :
    label_path = f"../../asset/{task}/test.csv"
    pred_path = f"../../asset/{task}/predictions/{model}.pkl"


    test_df = pd.read_csv(label_path, header=None, sep="\t")

    test_labels = test_df[0].values
    predicitons = load_pickle(pred_path)

    return accuracy(test_labels, predicitons)


task = "imdb"
model = "bert-base-uncased"


calculate_test_accuracy(task, model)


92.568

## Measuring the BTC that can be found by BiasFinder

In [10]:
mutation_tool = "biasfinder"
bias_type = "gender"
task = "imdb" # dataset used for fine-tuning
model = "bert-base-uncased"
# model = "bert-base-cased"
# model = "microsoft/mpnet-base"
mutant = "imdb" # dataset used for generating mutants

def load_mutant_and_prediction(mutation_tool, bias_type, mutant):
    base_dir = f"../../data/{mutation_tool}/{bias_type}/{mutant}/"
    if mutation_tool == "biasfinder" :
        df = pd.read_csv(base_dir + "test.csv", header=None, sep="\t", names=["label", "mutant", "template", "original", "gender"])
    elif mutation_tool == "eec":
        df = pd.read_csv(base_dir + "test.csv", header=None, sep="\t", names=["label", "mutant", "template", "original", "person", "gender", "emotion"])

    df["template"] = df["template"].astype("category")
    df["template_id"] = df["template"].cat.codes

    prediction_fpath = os.path.join(base_dir, f"predictions/{model}.pkl")
    pred = load_pickle(prediction_fpath)

    df["prediction"] = pred

    return df


df = load_mutant_and_prediction(mutation_tool, bias_type, mutant)

df.head()


,label,mutant,template,original,gender,template_id,prediction
0,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",male,929,0
1,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",female,929,0
2,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1
3,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1
4,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1


In [11]:
print("Accuracy on mutants: {:.2f}%".format(accuracy(df["label"],df["prediction"])))


Accuracy: 93.29%


In [5]:
def print_evaluation(evaluation):
    print("# Mutants \t:", evaluation["mutant"])
    print("# Templates \t:", evaluation["template"])
    print("# BTCs \t\t:", evaluation["btc"])

evaluation = evaluate_btc(df["label"], df["prediction"], df["mutant"], df["template"], "gender", df["gender"])
print_evaluation(evaluation)

# Mutants 	: 153866
# Templates 	: 3015
# BTCs 		: 5543


In [19]:
# models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base", "facebook/muppert-roberta-base", "google/electra-base-generator"]

# fine-tune
# 

# test accuracy
# "microsoft/deberta-base"

# predict mutants
# "bert-base-cased", "roberta-base", "albert-base-v2",


models = ["bert-base-uncased", "microsoft/mpnet-base"]

mutation_tool = "biasfinder"
bias_type = "gender"
task = "imdb"  # dataset used for fine-tuning
mutant = "imdb"  # dataset used for generating mutants

d = pd.DataFrame(columns=["tool", "model", "accuracy",
                          "template", "mutant", "btc"])

for model in models :
    test_accuracy = calculate_test_accuracy(task, model)
    df = load_mutant_and_prediction(mutation_tool, bias_type, mutant)
    evaluation = evaluate_btc(df["label"], df["prediction"],
                            df["mutant"], df["template"], "gender", df["gender"])

    d = d.append(
            {
                "model": model, 
                "accuracy": test_accuracy, 
                "tool": mutation_tool, 
                "template": evaluation['template'], 
                "mutant": evaluation['mutant'],
                "btc" : evaluation['btc']
            }, 
            ignore_index=True)


In [20]:
d

,model,accuracy,tool,template,mutant,btc
0,bert-base-uncased,92.568,biasfinder,3015,153866,5543
1,microsoft/mpnet-base,93.616,biasfinder,3015,153866,4849
